In [2]:
import tensorflow as tf

In [3]:
print(tf.__version__)   # Tutorial written on V1.15.0 ensure that you are using the same or anything compatible

1.12.0


# Creating A Dataflow Graph Using Tensorflow Variables

In these few lines, there are a lot of peculiarities of TensorFlow and its way of building a computational graph. 
- This graph represents the matrix product between the constant tensor identified by the A Python variable and the constant tensor identified by the x Python variable and the sum of the resulting matrix with the tensor identified by the b Python variable. 

- The result of the computation is represented by the y Python variable, also known as the output of the tf.add node named result in the graph.

In [4]:
A = tf.constant([[1,2], [3,4]], dtype=tf.float32)
X = tf.constant([[0,10], [2.5,4]], dtype=tf.float32)
b = tf.constant([[2,3], [3,4.5]], dtype=tf.float32)

In [5]:
Y = tf.add(tf.matmul(A,X), b,name="result")             # Y = AX+b

In [6]:
print(Y)

Tensor("result:0", shape=(2, 2), dtype=float32)


- Please note the separation between the concept of a Python variable and a node in the graph: we're using Python only to describe the graph; the name of the Python variable means nothing in the graph definition.

- Please note that we are just describing the graph—the calls to the TensorFlow API are just adding operations (nodes) and connections (edges) among them; there is no computation performed in this phase. 

- In TensorFlow 1.x, the following approach needs to be followed—static graph definition and execution, while this is no longer mandatory in 2.0.


In [7]:
writer = tf.summary.FileWriter("log/matmul", tf.get_default_graph())
writer.close()

To run the tensorboard
- tensorboard --logdir Learning_Tensorflow\log\matmul

# Defining Tensorflow Graph

- The following code snippet shows how our baseline example can be wrapped into a separate graph
- How a second independent graph can be created in the same Python script,
- how we can change the node names, adding a prefix, using tf.name_scope. 

In [8]:
g1 = tf.Graph()
g2 = tf.Graph()

In [9]:
with g1.as_default():
    A = tf.constant([[1,2], [3,4]], dtype=tf.float32)
    X = tf.constant([[2,4], [3.5,3.5]], dtype=tf.float32)
    b = tf.constant([[1,-1]], dtype=tf.float32)
    y = tf.add(tf.matmul(A,X), b, name="result")

In [10]:
with g2.as_default():
    with tf.name_scope("scope_a"):
        x = tf.constant(1, name="x")
        print(x)
    with tf.name_scope("scope_b"):
        x = tf.constant(2, name="x")
        print(x)
    y = tf.constant(12)
    z = x * y


Tensor("scope_a/x:0", shape=(), dtype=int32)
Tensor("scope_b/x:0", shape=(), dtype=int32)


- Then, we define two summary writers. We need to use two different tf.summary.FileWriter objects to log two separate graphs.

In [13]:
writer = tf.summary.FileWriter("log/two_graphs/g1", g1)
writer = tf.summary.FileWriter("log/two_graphs/g2",g2)
writer.close()

- Nodes with the same name, x in the example, can live together in the same graph, but they have to be under different scopes. 
- In fact, being under different scopes makes the nodes completely independent and completely different objects.

In general, every tensor has a name, a type, a rank, and a shape:

- The name uniquely identifies the tensor in the computational graphs. Using tf.name_scope, we can prefix tensor names, thus changing their full path. 

- The type is the data type of the tensor; for example, tf.float32, tf.int8, and so on.

- The rank is the number of dimensions of a tensor.

- The shape is the number of elements in each dimension; for example, a scalar has rank 0 and an empty shape of (), a vector has rank 1 and a shape of (D0), a matrix has rank 2 and a shape of (D0, D1), and so on. 


- Being a C++ library, TensorFlow is strictly statically typed. 
- This means that the type of every operation/tensor must be known at graph definition time. Moreover, this also means that it is not possible to execute an operation among incompatible types.

In [14]:
# Using operator overloading we can also write
A = tf.constant([[1,2], [3,4]], dtype=tf.float32)
X = tf.constant([[2,4], [3.5,3.5]], dtype=tf.float32)
b = tf.constant([[1,-1]], dtype=tf.float32)


In [15]:
y = A@X + b

In [16]:
print(Y)

Tensor("result:0", shape=(2, 2), dtype=float32)


# tf.device

- tf.device creates a context manager that matches a device. The function allows the user to request that all operations created within the context it creates are placed on the same device.

- The devices identified by tf.device are more than physical devices; in fact, it is capable of identifying devices such as remote servers, remote devices, remote workers, and different types of physical devices (GPUs, CPUs, and TPUs). 

- It is required to follow a device specification to correctly instruct the framework to use the desired device.

In [17]:
with tf.device("/CPU:0"):
    A = tf.constant([[1,2], [3,4]], dtype=tf.float32)
    B = tf.constant([[2,3], [-1,-2]], dtype=tf.float32)

In [18]:
# IF we have a GPU
with tf.device("/GPU:0"):
    mul = A @ B

In [19]:
writer = tf.summary.FileWriter("log/matmul_optimized", tf.get_default_graph())
writer.close()

- When using the static-graph and session execution parading, the execution is completely separated from the graph definition. This is no longer true in eager execution, 

# tf.session

- tf.Session is a class that TensorFlow provides to represent a connection between the Python program and the C++ runtime. 

- The tf.Session object is the only object able to communicate directly with the hardware (through the C++ runtime), placing operations on the specified devices, using the local and distributed TensorFlow runtime, with the goal of concretely building the defined graph.
 - The tf.Session object is highly optimized and, once correctly built, caches tf.Graph in order to speed up its execution.

tf.sessison object does these three functions

- Acquires resources.
- Use resources.
- Release resources. 

In [ ]:
# The context manager opens the session
with tf.Session() as sess:
    # Use the session to execute operations
    sess.run(...)
# Out of the context, the session is closed and the resources released

In [ ]:
# the IP and port of the TensorFlow server
ip = "192.168.1.90"
port = 9877
with tf.Session(f"grpc://{ip}:{port}") as sess:
    sess.run(...)

By default, the tf.Session will capture and use the default tf.Graph object. 

In [25]:
# with tf.device("/CPU:0"):
A = A = tf.constant([[1, 2], [3, 4]], dtype=tf.float32)
x = tf.constant([[0, 10], [0, 0.5]])
b = tf.constant([[1, -1]], dtype=tf.float32)
y = tf.add(tf.matmul(A, x), b, name="result")

In [26]:
writer = tf.summary.FileWriter("log/matmul",tf.get_default_graph())
writer.close()

In [28]:
with tf.Session() as sess:
    A_value, x_value, b_value =sess.run([A,x,b])
    y_value = sess.run(y)

    # Overwrite
    y_new = sess.run(y, feed_dict={b:np.zeros((1,2))})

print(f"A: {A_value}\nx: {x_value}\nb: {b_value}\n\ny: {y_value}")
print(f"y_new: {y_new}")


- The first sess.run call evaluates the three tf.Tensor objects, A, x, b, and returns their values as numpy arrays.
- The third sess.run call shows how it is possible to inject into the computational graph values from the outside, as numpy arrays, overwriting a node. The feed_dict parameter allows you to do this: usually, inputs are passed to the graph using the feed_dict parameter and through the overwriting of the tf.placeholder operation created exactly for this purpose.

- tf.placeholder is just a placeholder created with the aim of throwing an error when values from the outside are not injected inside the graph. However, the feed_dict parameter is more than just a way to feed the placeholders. In fact, the preceding example shows how it can be used to overwrite any node.

# Variables in Tensorflow

- A variable is an object that maintains a state in the graph across multiple calls to sess.run. A variable is added to tf.Graph by constructing an instance of the tf.Variable class.

- A variable is completely defined by the pair (type, shape), and variables created by calling tf.Variable can be used as input for other nodes in the graph; in fact, the tf.Tensor and tf.Variable objects can be used in the same manner when building a graph.

- Variables have more attributes with respect to tensors: a variable object must be initialized and thus have its initializer; a variable is, by default, added to the global variables and trainable variable graph collections. If a variable is set as non-trainable, it can be used by the graph to store the state, but the optimizers will ignore it when performing the learning process.

## tf.variable

- Creating a variable by calling tf.Variable will always create a new variable and it always requires an initial value to be specified. 
- Truncated Normal gives values in normal distribution.

In [29]:
size_in = 28
size_out = 28
w = tf.Variable(tf.truncated_normal([5,5,size_in,size_out], stddev=0.1), name = 'W')
b = tf.Variable(tf.constant(0.1, shape = [size_out]), name = "B")

In [30]:
print(w)
print(b)

<tf.Variable 'W:0' shape=(5, 5, 28, 28) dtype=float32_ref>
<tf.Variable 'B:0' shape=(28,) dtype=float32_ref>


Since each call to tf.Variable creates a new variable in the graph, it is the perfect candidate for the creation of layers

This creates a 5x5 Convolution Kernel

In [31]:
def Conv2D(input, size_in, size_out, name = "conv"):
    '''
    Creates a conv2D layer which we use frequrently.
    Input: a 4D tensor (batch_size, dim1, dim2, channels)
    size_in: Can be inferred from input.
    size_out: No. of kernels to learn.
    Ouput: Return of Conv2D operation applied + MaxPooling which halves the dimesnsion
    '''

    with tf.name_scope(name):
        w = tf.variable(tf.truncated_normal([5,5,size_in,size_out], stddev=0.1), name='W')
        b = tf.variable(tf.constant(0.1, shape=[size_out]), name="B")
        conv = tf.nn.Conv2D(input, w, strides = [1,1,1,1], padding='SAME')
        act = tf.nn.relu(conv + b)
        tf.summary.histogram("w",w)
        tf.summary.historgram("b",b)
        return(tf.nn.max_pool(act, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')) 


Defining a fully_connected layer

In [33]:
def fc(input, size_in, size_out, name="fc"):
    '''
    Args: Input 2D tensor
    size_in: it could be inferred by the input (input.shape[-1])
    size_out: the number of output neurons kernel to learn
    Output: Linear 1D output
    '''

    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([size_in,size_out], stddev=0.1), name="W")
        b = tf.Variable(tf.constant(0.1, shape = [size_out]), name="B")
        act = tf.matmul(input, w) + b
        tf.summary.histogram("w",w)
        tf.summary.histogram("b",b)
        return(act)
 


Both functions also use the tf.summary module to log the histograms of the weight, bias, and activation values, which can change during training.

The call to a tf.summary method automatically adds the summaries to a global collection that is used by tf.Saver and tf.SummaryWriter objects to log every summary value in the TensorBoard log directory

## tf.Get_variable

- To enable variable sharing where we need to use the same variable name for different layers.
- tf.get_variable is always used together with tf.variable_scope since it enables the variable sharing capabilities of tf.get_variable through its reuse parameter
- Hence, a layer that uses tf.get_variable to define variables can be used in conjunction with tf.variable_scope to define or reuse the layer's variables. 
- This is useful in GANs

In [39]:
with tf.variable_scope("scope", reuse=True):
    a = tf.get_variable("v", [1])
with tf.variable_scope("scope", reuse=True):
    b = tf.get_variable("v",[1])
with tf.variable_scope("scope", reuse=True):
    c = tf.get_variable("v",[1])

In [45]:
def Conv2D(input, size_in, size_out):
    w = tf.get_variable('W', [5,5,size_in,size_out], initializer = tf.truncated_normal_initializer(stddev=0.1))
    b = tf.get_variable('B', [size_out], initializer=tf.constant_initializer(0.1))
    conv = tf.nn.conv2d(input, w, strides = [1,1,1,1], padding = 'SAME')
    act = tf.nn.relu(conv+b)
    tf.summary.histogram("w",w)
    tf.summary.histogram("b",b)
    return(tf.nn.max_pool(act, ksize = [1,2,2,1], strides=[1,2,2,1], padding='SAME'))


In [46]:
def fc(input, size_in, size_out):
    w = tf.get_variable('W', [size_in,size_out], initializer = tf.truncated_normal_initializer(sttddev=0.1))
    b = tf.get_variable('B', [size_out], initializer=tf.constant_initializer(0.1))
    act = tf.matmul(input, w) + b
    tf.summary.histogram("w",W)
    tf.summary.histogram("b",b)
    return(act)

In [47]:
input = tf.placeholder(tf.float32, (None, 28,28,3))

In [50]:
with tf.variable_scope("first",reuse=True):
    conv1 = Conv2D(input, input.shape[-1].value, 10)

In [53]:
with tf.variable_scope("second",reuse=tf.AUTO_REUSE):
    conv2 = Conv2D(conv1, conv1.shape[-1].value, 1)

- TensorFlow already comes with a module named tf.layers, which contains all the most common and widely used layers, defined using tf.get_variable under the hood, and therefore, layers can be used in conjunction with tf.variable_scope to share their variables.

# tf.layers

**Disclaimer**
- tf.layers is completely removed in tensorflow 2.0
- We need to use tf.keras.layers instead.


The tf.layers module in TensorFlow 1.x and the tf.keras.layers module in TensorFlow 2.0 provide an excellent API to define machine learning models in a convenient and powerful way. 

Every layer in tf.layers, defines variables using tf.get_variable, and therefore, each layer defined in this way can use the variable-sharing features provided by tf.variable_scope.

Creating LeNet using tf.layers

In [63]:
def le_cnn(x, n_classes, reuse, is_training):
    """Defines a convolutional neural network for classification.
    Input: x: 4D Tensor (Batch_size, dim1, dim2, channels)
    n_classes: the number of classes, hence, the number of output neurons.
    reuse: the tf.variable_scope reuse parameter
    is_training: boolean variable indicates if we are training

    return: An output layer

    """

    # no. of filters = 64
    # size of kernel = 3x3
    conv1 = tf.layers.conv2d(x, 64, 3, activation=tf.nn.relu)
    conv1 = tf.layers.max_pooling2d(conv1, 2, 2)

    conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
    conv2 = tf.layers.max_pooling2d(conv2, 2, 2)

    # Flattening into 2D to connect with fc

    shape = (-1, conv2.shape[1].value * conv2.shape[2].value * conv2.shape[3].value)
    fc1 = tf.reshape(conv2, shape)

    # Fully connected layer
    fc1 = tf.layers.dense(fc1, 1024)
    out = tf.layers.dense(fc1, n_classes)
    return(out)


In [64]:
inputs = tf.placeholder(tf.float32, (None,28,28,1))
logits = le_cnn(input, 10, reuse=False, is_training = True)

# Automatic Differentiation and Training Network

- tf contains automatic differentiation.
- All the losses are in tf.losses
- Optimizers can be found in tf.train

In [66]:
labels = tf.placeholder(tf.int32, (None,))

In [67]:
# Loss function
loss = tf.losses.sparse_softmax_cross_entropy(labels, logits)
# Optimizer
opt = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

In [68]:
# logging the graph
writer = tf.summary.FileWriter("log/graph_loss", tf.get_default_graph())
writer.close()

- Python is used only to build a graph and to do non-learning related operations

# A working Example Fashion MNIST

In [76]:
from tensorflow.keras.datasets import fashion_mnist
import numpy as np

In [77]:
(train_x, train_y), (test_x, test_y) = fashion_mnist.load_data()

In [78]:
# To scale in [-1,1] range
train_x = (train_x / 255.) * 2 - 1
test_X = (test_x / 255.) * 2 - 1

In [79]:
# Adding 1 last dimenstion to make it in (28,28,1)
train_x = np.expand_dims(train_x,-1)
test_x = np.expand_dims(test_x,-1)

In [80]:
epochs = 10
batch_size = 32
number_batches_train = int(train_x.shape[0] / batch_size)

In [83]:
print("Batch size is %d" %(batch_size))
print("Number of batches per epochs = %d" %(number_batches_train))

Batch size is 32
Number of batches per epochs = 1875


## Defining Accuracy

In [87]:
# predictions = tf.argmax(logits, 1)
# # correct predictions: [BATCH_SIZE] tensor
# correct_predictions = tf.equal(labels, predictions)
# accuracy = tf.redeuce_mean(tf.cast(correct_predictions, tf.float32), name = "accuracy")

In [ ]:
# accuracy_summary = tf.summary.scalar("accuracy", accuracy)
# loss_summary = tf.summary.scalar("loss", loss)

In [ ]:
writer = tf.summary.FileWriter("log/graph_loss", tf.get_default_graph())
validation_summary_writer = tf.summary.FileWriter("log/graph_loss/validation")

# Training Example

- tf.Saver is the object the TensorFlow Python API provides to save the current model variables. Please note that the tf.Saver object saves the variables only and not the graph structure!
- To save both the graph structure and variables, a SavedModel object is required;

In [95]:
def train():
    # tf.device("CPU:0")
    input = tf.placeholder(tf.float32, (None, 28, 28, 1))
    labels = tf.placeholder(tf.int64, (None,))
    logits = le_cnn(input, 10, reuse=False, is_training=True)
    loss = tf.losses.sparse_softmax_cross_entropy(labels, logits)
    global_step = tf.train.get_or_create_global_step()
    train_op = tf.train.AdamOptimizer().minimize(loss, global_step)

    writer = tf.summary.FileWriter("log/graph_loss", tf.get_default_graph())
    validation_summary_writer = tf.summary.FileWriter(
        "log/graph_loss/validation")

    init_op = tf.global_variables_initializer()

    predictions = tf.argmax(logits, 1)
    # correct predictions: [BATCH_SIZE] tensor
    # tf.metrics.accuracy(predictions, train_y)
    
    correct_predictions = tf.equal(labels, predictions)
    accuracy = tf.reduce_mean(
        tf.cast(correct_predictions, tf.float32), name="accuracy")

    accuracy_summary = tf.summary.scalar("accuracy", accuracy)
    loss_summary = tf.summary.scalar("loss", loss)
    # Input preprocessing a Python stuff
    (train_x, train_y), (test_x, test_y) = fashion_mnist.load_data()
    # Scale input in [-1, 1] range
    train_x = train_x / 255. * 2 - 1
    train_x = np.expand_dims(train_x, -1)
    test_x = test_x / 255. * 2 - 1
    test_x = np.expand_dims(test_x, -1)

    epochs = 10
    batch_size = 32
    nr_batches_train = int(train_x.shape[0] / batch_size)
    print(f"Batch size: {batch_size}")
    print(f"Number of batches per epoch: {nr_batches_train}")

    validation_accuracy = 0
    saver = tf.train.Saver()
    with tf.Session() as sess:
        sess.run(init_op)

        for epoch in range(epochs):
            for t in range(nr_batches_train):
                start_from = t * batch_size
                to = (t + 1) * batch_size

                loss_value, _, step = sess.run(
                    [loss, train_op, global_step],
                    feed_dict={
                        input: train_x[start_from:to],
                        labels: train_y[start_from:to]
                    })
                if t % 10 == 0:
                    print(f"{step}: {loss_value}")
            print(
                f"Epoch {epoch} terminated: measuring metrics and logging summaries"
            )

            saver.save(sess, "log/graph_loss/model")
            start_from = 0
            to = 128
            train_accuracy_summary, train_loss_summary = sess.run(
                [accuracy_summary, loss_summary],
                feed_dict={
                    input: train_x[start_from:to],
                    labels: train_y[start_from:to]
                })

            validation_accuracy_summary, validation_accuracy_value, validation_loss_summary = sess.run(
                [accuracy_summary, accuracy, loss_summary],
                feed_dict={
                    input: test_x[start_from:to],
                    labels: test_y[start_from:to]
                })

            # save values in TensorBoard
            writer.add_summary(train_accuracy_summary, step)
            writer.add_summary(train_loss_summary, step)

            validation_summary_writer.add_summary(validation_accuracy_summary,
                                                  step)
            validation_summary_writer.add_summary(validation_loss_summary, step)

            validation_summary_writer.flush()
            writer.flush()

            # model selection
            if validation_accuracy_value > validation_accuracy:
                validation_accuracy = validation_accuracy_value
                saver.save(sess, "log/graph_loss/best_model/best")

    writer.close()

In [96]:
train()

Batch size: 32
Number of batches per epoch: 1875


InvalidArgumentError: Cannot assign a device for operation matmul_2: Operation was explicitly assigned to /device:GPU:0 but available devices are [ /job:localhost/replica:0/task:0/device:CPU:0 ]. Make sure the device specification refers to a valid device.
	 [[node matmul_2 (defined at <ipython-input-18-41fef96e24eb>:3)  = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/device:GPU:0"](Const_6, Const_7)]]

Caused by op 'matmul_2', defined at:
  File "C:\Program Files\Python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Program Files\Python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Program Files\Python36\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Program Files\Python36\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Program Files\Python36\lib\site-packages\ipykernel\kernelap